In [ ]:
!pip3 install numpy matplotlib scipy scikit-learn
!pip3 install keras
!pip3 install pandas
!pip3 install os-sys

In [ ]:
!pip3 install nltk spacy

In [4]:
import fnmatch
import os
import zipfile
import re

import numpy as np
from numpy.lib import stride_tricks
import os
import tarfile

import pandas as pd

import random

import nltk
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data

/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data


In [7]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data'

# directory where audio and transcripts folders will be created
out_dir = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw'

In [8]:
# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv')

# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv')

# /content/drive/MyDrive/Depression_Analysis_Research/dev_split.csv
dev_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/dev_split.csv')

In [9]:
print(train_df.shape)
print(test_df.shape)
print(dev_df.shape)

(163, 6)
(56, 6)
(56, 6)


In [10]:
print('train_df.head()', train_df.head())
print('test_df.head()', test_df.head())
print('dev_df.head()', dev_df.head())

train_df.head()    Participant_ID   Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             302     male           0          4             0             28
1             303   female           0          0             0             17
2             304   female           0          6             0             20
3             305     male           0          7             0             28
4             307  female            0          4             0             23
test_df.head()    Participant_ID  Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             600  female           0          5             0           23.0
1             602  female           1         13             1           67.0
2             604    male           1         12             0           30.0
3             605    male           0          2             0           23.0
4             606  female           0          5             0           46.0
dev_df.head()    Participan

In [11]:
def in_split(partic_id, df):
    """
    Returns True if the participant is in the AVEC training split
    (aka participant's we have depression labels for)
    """
    return partic_id in set(df['Participant_ID'].values)

def get_depression_label(partic_id, df):
    """
    Returns participant's PHQ8 Binary label. 1 representing depression;
    0 representing no depression.
    """
    return df.loc[df['Participant_ID'] == partic_id]['PHQ_Score'].item()

In [46]:
def build_class_dictionaries(dir_name, df):
    
    print("dir name: " + dir_name)

    depressed_dict = dict()
    normal_dict = dict()
    
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            # print(file)
            if fnmatch.fnmatch(file, '*_Transcript.csv'):
                # print(file)
                regex = re.compile(r'\d+')
                partic_id = [int(x) for x in regex.findall(file)][0]
                # print(partic_id)
                if in_split(partic_id, df):
                    file_name = dir_name + '/' + str(partic_id) + '_P/' + file
                    
                    df_1 = pd.read_csv(file_name, header=None, delimiter=',')
                    df_1.drop([0], axis=1, inplace=True)
                    mat = np.mat(df_1)

                    depressed = get_depression_label(partic_id, df)  # 1 if True
                    if depressed:
                        # print("Depressed")
                        # print(partic_id)
                        # print(mat)
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
    return depressed_dict, normal_dict

In [47]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/transcripts'
depressed_train_dict, normal_train_dict = build_class_dictionaries(dir_name, train_df)
# depressed_train_dict

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/transcripts


In [69]:
count = 0
for key, val in depressed_train_dict.items():
    print(key)
    
    break
    # df = pd.DataFrame(list(zip([ls for ls in val])),
    #     columns = [0,1,2])

302
<class 'numpy.matrix'>
